### Libraries

In [1]:
import numpy as np 
import pandas as pd

### Reading in the data from the internet

In [2]:
# Read in the data from the online spreadsheet
EXPERIMENTS = pd.read_csv('https://docs.google.com/spreadsheets/d/'+
                          '1ivpOBFxBW_3guvNMOkRfU7e-hFXRtjXCTbr4aTJcPdY/' +
                          'export?gid=0&format=csv',
                          header=0,
                          names=['Experiment', 'Telescope', 'Publications', 'Location',
                                 'Type', 'Antenna type', 'Frequency (MHz)', 'Status'],)
# Convert the data to the correct types
EXPERIMENTS['Frequency (MHz)'] = EXPERIMENTS['Frequency (MHz)'].apply(lambda x: np.array(x.split(',')).astype(np.float64) if x == x else x)
EXPERIMENTS['Publications'] = EXPERIMENTS['Publications'].apply(lambda x: x.split(',') if x == x else x)
EXPERIMENTS['Location'] = EXPERIMENTS['Location'].apply(lambda x: x.split(';') if x == x else x)
    
# Define a function that filters the data table based on the experiment
def get_experiment(experiment_name):
    experiment_info = EXPERIMENTS[EXPERIMENTS['Experiment'] == experiment_name]
    if len(experiment_info) == 0:
        raise ValueError(f"Experiment {experiment_name} not found.")
    else:
        return experiment_info
    
# Define a function that returns the redshift range of the experiment
def get_redshift(experiment_name):
    experiment_info = get_experiment(experiment_name)
    freq21cm = 1420.40575177
    frequency = experiment_info['Frequency (MHz)']
    z_info = experiment_info[['Telescope', 'Frequency (MHz)']].copy()
    z_info['Redshift'] = ((freq21cm/frequency) - 1).apply(lambda x: np.round(x, 2))
    return z_info

### Using the data-frame

In [3]:
get_experiment('SARAS')

Experiment Telescope                                 Publications  \
0      SARAS    SARAS1                        [2013ExA....36..319P]   
1      SARAS    SARAS2  [2018ExA....45..269S,  2017ApJ...845L..12S]   
2      SARAS    SARAS3  [2021arXiv210401756N,  2020JAI.....950006G]   

                                      Location    Type  \
0  [Gauribidanur Observatory (southern India)]  global   
1       [Timbaktu Collective (southern India)]  global   
2                             [Southern India]  global   

                          Antenna type Frequency (MHz)  Status  
0  Fat-dipole correlation spectrometer   [87.5, 175.0]     NaN  
1                             Monopole  [110.0, 200.0]     NaN  
2                             Monopole   [40.0, 200.0]     NaN

In [4]:
get_redshift('SARAS')

Telescope Frequency (MHz)       Redshift
0    SARAS1   [87.5, 175.0]  [15.23, 7.12]
1    SARAS2  [110.0, 200.0]   [11.91, 6.1]
2    SARAS3   [40.0, 200.0]   [34.51, 6.1]

### Generating TeX tables

In [5]:
# Get the information for the SARAS experiment
SARAS_info = get_experiment('SARAS').copy()
# Modify columns as needed
SARAS_info['Publications'] = SARAS_info['Publications'].apply(lambda x: '\\citep{' + ','.join(x) + '}' if x == x else x)
# Convert the data to LaTeX
SARAS_tex = SARAS_info.to_latex(columns=['Telescope', 'Publications', 'Frequency (MHz)', 'Location'], index=False)
print(SARAS_tex)

\begin{tabular}{llll}
\toprule
Telescope & Publications & Location & Frequency (MHz) \\
\midrule
SARAS1 & \citep{2013ExA....36..319P} & ['Gauribidanur Observatory (southern India)'] & [ 87.5 175. ] \\
SARAS2 & \citep{2018ExA....45..269S, 2017ApJ...845L..12S} & ['Timbaktu Collective (southern India)'] & [110. 200.] \\
SARAS3 & \citep{2021arXiv210401756N, 2020JAI.....950006G} & ['Southern India'] & [ 40. 200.] \\
\bottomrule
\end{tabular}

